In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os, sys
sys.path.append('..')
from glob import glob
from configs import *
from tqdm import tqdm
from utils import *
import numpy as np
from time import time
tqdm.monitor_interval = 0
import json
import numpy as np
from time import time, strptime, mktime
import matplotlib.pyplot as plt

In [4]:
sfiles = glob('../data/common_stops/*.txt')
len(sfiles)

40

In [58]:
vs = []
adjlist = {}

scounts = {}
for fname in sfiles:
    with open(fname) as fl:
        stops = fl.read().split('\n')
#     segs = []
    for  stop in stops:
        if stop not in vs: vs.append(stop)
    for si in range(1, len(stops)):
        if stops[si-1] not in adjlist: adjlist[stops[si-1]] = []
        if stops[si] not in adjlist[stops[si-1]]: adjlist[stops[si-1]].append(stops[si])
        skey = '%s-%s' % (stops[si-1], stops[si])
#         segs.append(skey)
        if skey not in scounts:
            scounts[skey] = 0
        scounts[skey] += 1
        
skeys = sorted(list(scounts.keys()), key=lambda k: scounts[k])
for seg in reversed(skeys):
    print(seg, scounts[seg])

400042-400043 4
400043-400044 4
400044-400045 4
400041-400042 4
404191-400041 3
400030-405392 3
400360-400361 3
402816-403749 3
402831-403422 3
400361-400362 3
803163-403424 3
402856-404106 3
401458-401459 3
402841-402842 3
403821-403822 3
402829-403421 3
402842-402843 3
401974-402816 3
400359-400360 3
400037-400038 3
400026-404994 3
400028-400029 3
402852-405272 3
400038-400039 3
401973-401974 3
404321-402826 3
405392-400032 3
403419-404321 3
400363-400364 3
400045-400046 3
400034-400036 3
400033-400034 3
405066-403419 3
400860-400861 3
402851-402852 3
400224-400225 3
402845-402846 3
403421-402831 3
403424-402841 3
400036-400037 3
403456-403457 3
403749-403418 3
403417-401973 3
402838-803163 3
402854-403772 3
400364-400026 3
400032-400033 3
400029-400030 3
400561-400358 3
402819-404947 3
405265-405152 3
400111-404081 3
403267-400224 3
403425-402845 3
400110-400111 3
404947-405066 3
400039-404191 3
400046-404856 3
404994-400028 3
404854-402829 3
404856-403267 3
402826-404854 3
400362-4

In [51]:
gobj = show_graph(vs, adjlist, vdesc=lambda seg: seg[0] + seg[-2:])
gobj.render('stops')

'stops.jpg'

In [64]:
valids = {}
with open('../Valid-Counts-Sorted.txt') as fl:
    lines = fl.read().split('\n')[1:-1]
for ln in lines:
#     print(ln)
    seg, nn = ln.split(',')
    nn = int(nn)
    valids[seg] = nn

In [83]:
def s2g(segs):
    vs = segs
    adjlist = {}
    for ii, fr in enumerate(vs):
        if fr not in adjlist:
            adjlist[fr] = []
        for jj, to in enumerate(vs):
            if ii == jj:
                continue
            if fr.split('-')[1] == to.split('-')[0]:
                if to not in adjlist[fr]: adjlist[fr].append(to)
                
#                 if to not in adjlist: adjlist[to] = []
#                 if fr not in adjlist[to]: adjlist[to].append(fr)
    
    no_singles = []
    for vert in vs:
        if vert not in adjlist or len(adjlist[vert]) == 0:
            hasparent = False
            for _, ls in adjlist.items():
                if vert in ls:
                    hasparent = True
            if not hasparent:
                if vert in adjlist:
                    del adjlist[vert]
                continue
        no_singles.append(vert)
    return no_singles, adjlist

In [61]:
def seg2val(seg):
    if seg in valids:
        return str(int(valids[seg] / 13248 * 100))
    else:
        return '?'

vs, adjlist = s2g(skeys)
gobj = show_graph(vs, adjlist, vdesc=seg2val)
gobj.render('graph')

'graph.jpg'

In [70]:
high_avail = [seg for seg in skeys if (seg in valids and (valids[seg]/13248) > 0.5 )]
print(len(high_avail))

709


In [84]:
vs, adjlist = s2g(high_avail)
gobj = show_graph(vs, adjlist, vdesc=seg2val)
gobj.render('avail')

'avail.jpg'

In [85]:
cvs, cadj = complete_graph(vs, adjlist)

In [87]:
with open('../data/man.json', 'w') as fl:
    json.dump([
        vs,
        adjlist
    ], fl, indent=4)

### Organize by hops